AMSR VS CYGNSS without interpolation

In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import array
from scipy.interpolate import RegularGridInterpolator
import sys
import math

In [758]:
def main(day):
    # print('in')
    fn      = '/glade/work/geethma/research/data_cygnss/2019/cyg.ddmi.s201901'+f'{day:02d}'+'-003000-e201901'+f'{day:02d}'+'-233000.l3.grid-wind.a30.d31.nc'
    dsi     = nc.Dataset(fn)
    # ds.append(dsi)
    lons = dsi.variables['lon'][:]           #size=1800 #Range is 0.1 .. 359.9
    lats = dsi.variables['lat'][:]        #size=400 #Range is -39.9 .. 39.9
    globals()['u'] = dsi.variables['wind_speed'][:,:,:]    #float wind_speed(time, lat, lon) #units = "m s-1"
    globals()['time']    = dsi.variables['time'][:]          #size=24
    dsi.close
##############
    fna     = '/glade/work/geethma/research/data_amsr2/2019/RSS_AMSR2_ocean_L3_daily_2019-01-'+f'{day:02d}'+'_v08.2.nc'
    dsa     = nc.Dataset(fna)
    lonsa   = dsa.variables['lon'][:]          #size=1440  #Range is 0.125 .. 359.875
    latsa   = dsa.variables['lat'][:]          #size=720 #Range is -89.875 .. 89.875
    globals()['ua']      = dsa.variables['wind_speed_AW'][:,:,:]  #float32 wind_speed_LF(pass, lat, lon)
    globals()['timea']   = dsa.variables['time'][:,:,:]            #(pass, lat, lon) shape=(2,720,1440)
    globals()['passa']   = dsa.variables['pass'][:]        #[1 2]
    raina   = dsa.variables['rain_rate'][:,:,:]
    dsa.close

    globals()['lons'] = lons
    globals()['lonsa'] = lonsa
    globals()['lats'] = lats
    globals()['latsa'] = latsa
    
    

import numpy.ma as ma
x = ma.array(u)
Y = x[~x.mask]
print(np.shape(Y))

In [759]:
para = 10
main(para)
# ind(i)
# t(i)
# c(i)
# a(i)
# app(i)
# coord(i)
# save(i)
    


In [760]:

lon_ind = []
lon_inda = []
lat_ind = []
lat_inda = []


for i in range(len(lons)):
    for j in range(len(lonsa)):
        if (abs(lonsa[j]-lons[i])<=0.025):
            lon_ind.append(i)
            lon_inda.append(j)

for i in range(len(lats)):
    for j in range(len(latsa)):
        if (abs(latsa[j]-lats[i])<=0.025):
            lat_ind.append(i)
            lat_inda.append(j)



# np.savez_compressed('/glade/work/geethma/research/npzfilesn/2019_/january/near_01012019', cw=cyg, aw=am, b=bias, ar=r, cT=cT, aT=aT, lon=longt, lat=latit) #u10_monthdateyear.npz


In [761]:

a_time = np.empty((2,len(lat_inda),len(lon_inda)))
z = 0
for i in (lon_inda):
    y = 0
    for j in (lat_inda):
        x= 0
        for k in range(2):
#             a_time[x,y,z] = np.rint(timea[k,j,i])
            a_time[x,y,z] = timea[k,j,i]
#             print(x,y,z,a_time[x,y,z],timea[k,j,i],'e')
            x = x+1
        y = y+1
    z = z+1

/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Warning: converting a masked element to nan.
  if __name__ == '__main__':


In [762]:

cyg_wind = np.empty((24,len(lat_ind),len(lon_ind)))
z = 0
for i in (lon_ind):
    y = 0
    for j in (lat_ind):
        x= 0
        for k in range(24):
#             x = 0
            cyg_wind[x,y,z] == u[k,j,i]
#             print(cyg_wind[x,y,z])
            x = x+1
        y = y+1
    z = z+1

In [763]:

am_wind = np.empty((2,len(lat_inda),len(lon_inda)))
z = 0
for i in (lon_inda):
    y = 0
    for j in (lat_inda):
        x= 0
        for k in range(2):
#             x = 0
            am_wind[x,y,z] == ua[k,j,i]
            x = x+1
        y = y+1
    z = z+1

In [764]:
#cyg_wind[np.isnan(cyg_wind)] = 0
#am_wind[np.isnan(am_wind)] = 0

In [765]:

c = 0
cyg =[]
am=[]
bias=[]
for i in range(len(lon_ind)):
    for j in range(len(lat_inda)):
        for m in range(2):
            for n in range(0,24):
                if ((cyg_wind[n,j,i]>0.5) and (cyg_wind[n,j,i]<25)):
                    if ((am_wind[m,j,i]>0.5) and (am_wind[m,j,i]<25)):
#                     print(a_time[m,j,i],time[n])
                        if abs(a_time[m,j,i]-time[n])<0.5:
                            cyg.append(cyg_wind[n,j,i])
                            am.append(am_wind[m,j,i])
                            bias[latsa[lat_inda],lons[lon_ind]]=(cyg_wind[n,j,i] - am_wind[m,j,i])
    #                         print(a_time[m,j,i],time[n],m,j,i,cyg_wind[n,j,i],am_wind[m,j,i])
                            c=c+1
#                         print(timea[m,j,i],time[n])

print(c)
# globals()['cyg'] = cyg
# globals()['am'] = am
# globals()['bias'] = bias

0


In [766]:

longta = lonsa[lon_inda]
longt = lons[lon_ind]
latita = latsa[lat_inda]
latit = lats[lat_ind]

In [767]:
def save(day):
    np.savez_compressed('/glade/work/geethma/research/npzfilesn/2019_/january_/final_01'+f'{day:02d}'+'2019', cw=cyg, aw=am, b=bias, lon=longt, lat=latit,lona=longta, lata=latita) #u10_monthdateyear.npz


In [768]:
save(para)

# for i in range(3,31):
#     main(i)
#     ind(i)
#     t(i)
#     c(i)
#     a(i)
#     app(i)
#     coord(i)
#     save(i)
    


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
#####cw=cyg, aw=am, b=bias, lon=longt, lat=latit,lona=longta, lata=latita

fn = glob.glob('/glade/work/geethma/research/npzfilesn/2019_/january_/final*npz')
fn = np.array(fn)

In [ ]:
awf=[]
cwf = []
bf = []
# print(fn)
for i in range(0,len(fn)): #len(fn)
    # print (i)
    d = np.load(fn[i])
    aw = np.array(d['aw'])
    cw = np.array(d['cw'])
    bi = np.array(d['b'])
#     print(aw,'cyg',cw,'bias',bi)
    ##################
    awf.extend(aw)
    cwf.extend(cw)
    bf.extend(bi)
#     for i in range(0,len(aw)):
#         awf.append(aw[i])
#         cwf.append(cw[i])
#         bf.append(b[i])

np.mean(bf)

fig = plt.figure(figsize=(12,6))
plt.scatter(cwf, awf)
coefficients = np.polyfit(cwf, awf, 1)
m = round(coefficients[0],2)
c = round(coefficients[1],2)
poly = np.poly1d(coefficients)
new_x = np.linspace(min(cwf), max(cwf))
new_y = poly(new_x)
x = [0,17.5]
y = [0,17.5]
plt.plot(x,y,c='r',label='ideal mapping')
plt.plot(new_x,new_y,c='g',label='poly fit')
plt.legend()
plt.text(12, 4, 'gradient = '+str(m))
plt.text(12, 3, 'intercept = '+str(c)+' ms-1')
# # print(m, c)
plt.title('AMSR vs CYGNSS Wind Speeds on Jan 2019')
plt.xlabel('CYGNSS (m/s)')
plt.ylabel('AMSR (m/s)')
plt.show()

min(bf)

max(cwf)